# Overview of what we want to achieve:

* develop replacement model
* develop way to sample from the replacement model
* need to make a dataset and dataloader
* write the evaluation + calculate the accuracy
* need to put this as a Validation Metric to track in our training loop

## What are the different implementation options?

1. Combine gpt2 + CLT; use hooks
* we load gpt2 in memory
* we load (or have) the CLT in memory
* we use nnsight to put a lot of hooks into gpt2
* each hook runs part of the clt

2. write a replacement model class from scratch
* basically like a transformer but replace the MLPs with they weights from the clt
* read in attention weights, W_enc, W_pos, W_unemb from gpt2
* read in CLT weights from the CLT

### Things to consider for decision:
* easy to read and understand
* whether it's okay to ie during validation offload the CLT and gpt2 to load this model


### local replacement model
* run gpt2 on some prompts, store mlp_in, mlp_out
* run through the CLT model using activations from gpt2
* calculate the error terms recons - mlp_out

### What is the replacement model used for?
* just for evaluation

## Conclusion:

Since the replacement model will only be used as a validation metric and has no use cases downstream, the best option to implement this as a hook function model with nnsight because we won't need to load in weights, we can simply use the model

In [ ]:
print('hi')

In [ ]:
import torch


class ReplacementModel(torch.nn.Module):
    def __init__(self, gpt2, clt):
        super(ReplacementModel, self).__init__()
        self.gpt2 = gpt2
        self.clt = clt

    def overwrite_mlp_out(self, tokens):
        with self.gpt2.trace(tokens) as tracer:
            mlp_ins = []
            mlp_outs = []
            for i in range(12):
                mlp_in = self.gpt2.transformer.h[i].ln_2.input.save()
                mlp_ins.append(mlp_in)
                mlp_out = self.gpt2.transformer.h[i].mlp.output.save()
                self.gpt2.transformer.h[i].mlp.output = torch.zeros_like(mlp_out)
                mlp_outs.append(mlp_out)
        # batch layer in/out d_model
        mlp_ins = torch.stack(mlp_ins, dim=2)
        mlp_outs = torch.stack(mlp_outs, dim=2)
        mlp_acts = torch.stack([mlp_ins, mlp_outs], dim=2)
        return mlp_acts  # batch seq_len in/out n_layer d_model
    
    def forward(self, tokens):


In [ ]:
import nnsight

gpt2 = nnsight.LanguageModel('openai-community/gpt2', device_map='auto', dispatch=True)
gpt2.require_grad_(False)

: 

In [ ]:
dataset = datasets.load_dataset('Skylion007/openwebtext', split='train')
token_dataset = text_dataset.TextDataset(
    dataset,
    model.tokenizer,
    40,
    drop_last_batch=False,
    seq_len=1023,
)
text_dataset_loader = iter(
    DataLoader(
        token_dataset,
        batch_size=None,
        shuffle=False,
        num_workers=5,
        prefetch_factor=5,
        worker_init_fn=text_dataset.worker_init_fn,
    )
)

: 

In [ ]:
for batch in text_dataset_loader:

    break